<a href="https://colab.research.google.com/github/abdurrahmanrussel/chikungunya-india-visualization/blob/main/cikungunia_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geopandas folium matplotlib imageio openpyxl

from google.colab import files

# Upload Excel
print("Upload your Excel file (with Suspected and Confirmed cases):")
uploaded_excel = files.upload()
excel_file = list(uploaded_excel.keys())[0]

# Upload GeoJSON
print("Upload your GeoJSON file (Indian States):")
uploaded_geo = files.upload()
geo_file = list(uploaded_geo.keys())[0]


Upload your Excel file (with Suspected and Confirmed cases):


Saving fresh.xlsx to fresh.xlsx
Upload your GeoJSON file (Indian States):


Saving Indian_States.txt to Indian_States.txt


In [3]:
import pandas as pd

# Read Excel with two header rows
df = pd.read_excel(excel_file, header=[0,1])

# Flatten multi-index columns
df.columns = ['_'.join([str(i).strip() for i in col if i]).strip() for col in df.columns.values]

# Rename 'Affected States/UTs' column to simple name
for col in df.columns:
    if 'Affected States' in col:
        df.rename(columns={col:'Affected States/UTs'}, inplace=True)
        break

# Strip spaces
df['Affected States/UTs'] = df['Affected States/UTs'].str.strip()

print("Preview of Excel:")
print(df.head())


Preview of Excel:
   Sl. No._Unnamed: 0_level_1 Affected States/UTs 2018_Susp. cases  \
0                           1      Andhra Pradesh              622   
1                           2       Arunachal Pd.              507   
2                           3               Assam                3   
3                           4               Bihar              156   
4                           5        Chhattisgarh                0   

  2018_Conf. cases 2019_Susp. cases 2019_Conf. cases 2020_Susp. cases  \
0               79              832               88              318   
1                1              332               55                0   
2                3                0                0                0   
3              156              594              594               38   
4                0                0                0                0   

  2020_Conf. cases 2021_Susp. cases 2021_Conf. cases 2022_Susp. cases  \
0               28              753              

In [4]:
years = ['2018','2019','2020','2021','2022','2023','2024','2025']
sus_columns = {}
conf_columns = {}

for col in df.columns:
    for year in years:
        if col.startswith(year) and 'Susp' in col:
            sus_columns[year] = col
        elif col.startswith(year) and 'Conf' in col:
            conf_columns[year] = col

print("Suspected year columns:", sus_columns)
print("Confirmed year columns:", conf_columns)


Suspected year columns: {'2018': '2018_Susp. cases', '2019': '2019_Susp. cases', '2020': '2020_Susp. cases', '2021': '2021_Susp. cases', '2022': '2022_Susp. cases', '2023': '2023_Susp. cases', '2024': '2024_Susp. cases', '2025': '2025_Susp. cases'}
Confirmed year columns: {'2018': '2018_Conf. cases', '2019': '2019_Conf. cases', '2020': '2020_Conf. cases', '2021': '2021_Conf. cases', '2022': '2022_Conf. cases', '2023': '2023_Conf. cases', '2024': '2024_Conf. cases', '2025': '2025_Conf. cases'}


In [5]:
import geopandas as gpd

india = gpd.read_file(geo_file)
geo_states = india['NAME_1'].tolist()

state_mapping = {
    'Arunachal Pd.': 'Arunachal Pradesh',
    'Madhya Pd.': 'Madhya Pradesh',
    'Mizroram': 'Mizoram',
    'Odisha': 'Orissa',
    'Uttarakhand': 'Uttaranchal',
    'West Bengal**': 'West Bengal',
    'A& N Island': 'Andaman and Nicobar',
    'D&N Haveli': 'Dadra and Nagar Haveli',
    'Puduchery': 'Puducherry',
    'J&K': 'Jammu and Kashmir',
}

df['Affected States/UTs'] = df['Affected States/UTs'].replace(state_mapping)

# Check for missing states
excel_states = df['Affected States/UTs'].tolist()
missing_states = [state for state in geo_states if state not in excel_states]
print("Missing states in Excel:", missing_states)


Missing states in Excel: ['Daman and Diu']


In [7]:
# Create a DataFrame for missing states with all 0 values
missing_states_df = pd.DataFrame([
    {'Affected States/UTs': 'Daman and Diu', **{col: 0 for col in df.columns if col != 'Affected States/UTs'}}
])

# Concatenate with original DataFrame
df = pd.concat([df, missing_states_df], ignore_index=True)

# Verify
print("All states now in Excel:")
print(df['Affected States/UTs'].tolist())


All states now in Excel:
['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Meghalaya', 'Mizoram', 'Maharashtra', 'Manipur', 'Nagaland', 'Orissa', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttaranchal', 'West Bengal', 'Andaman and Nicobar', 'Chandigarh', 'Dadra and Nagar Haveli', 'Delhi', 'Jammu and Kashmir', 'Lakshadweep', 'Puducherry', 'Daman and Diu']


In [25]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from PIL import Image
import numpy as np
import imageio.v2 as imageio

def create_gif(df, year_columns, india, title_prefix, filename_gif):
    cities = {
        'New Delhi': (77.2090, 28.6139),
        'Mumbai': (72.8777, 19.0760),
        'Bengaluru': (77.5946, 12.9716),
        'Kolkata': (88.3639, 22.5726)
    }
    frames = []

    for year, col_name in year_columns.items():
        print(f"Processing {title_prefix} cases for {year}")
        merged = india.merge(
            df[['Affected States/UTs', col_name]],
            left_on='NAME_1', right_on='Affected States/UTs',
            how='left'
        )

        # Ensure numeric and fill missing
        merged[col_name] = pd.to_numeric(merged[col_name], errors='coerce').fillna(0).astype(int)

        fig, ax = plt.subplots(1, 1, figsize=(12,12))

        merged.plot(
            column=col_name,
            cmap='Purples',
            linewidth=0.8,
            ax=ax,
            edgecolor='0.8'
        )

        # Cities
        for city, (lon, lat) in cities.items():
            ax.plot(lon, lat, marker='o', color='red', markersize=6)
            ax.text(lon + 0.5, lat + 0.5, city, fontsize=12, color='black')

        ax.set_title(f'{title_prefix} Chikungunya Cases in India - {year}', fontsize=16)
        ax.axis('off')

        # Legend
        unique_vals = sorted(merged[col_name].unique())
        max_val = merged[col_name].max() if merged[col_name].max() > 0 else 1
        patches = [Patch(facecolor=plt.cm.Purples(val/max_val) if val>0 else 'white',
                         edgecolor='black', label=str(int(val))) for val in unique_vals]
        ax.legend(handles=patches, title=f'{title_prefix} Cases', loc='center left', bbox_to_anchor=(1,0.5), fontsize=10)

        # Save frame
        tmp_file = f'frame_{title_prefix}_{year}.png'
        plt.savefig(tmp_file, bbox_inches='tight', dpi=150)
        plt.close()

        img = Image.open(tmp_file).convert('RGB').resize((800,1000))
        frames.append(np.array(img))

    # Save GIF
    imageio.mimsave(filename_gif, frames, duration=[2000] * len(frames))
    print(f"GIF created: {filename_gif}")


In [26]:
create_gif(df, conf_columns, india, 'Confirmed', 'confirmed_chikungunya_cases.gif')
create_gif(df, sus_columns, india, 'Suspected', 'suspected_chikungunya_cases.gif')


Processing Confirmed cases for 2018
Processing Confirmed cases for 2019
Processing Confirmed cases for 2020
Processing Confirmed cases for 2021
Processing Confirmed cases for 2022
Processing Confirmed cases for 2023
Processing Confirmed cases for 2024
Processing Confirmed cases for 2025
GIF created: confirmed_chikungunya_cases.gif
Processing Suspected cases for 2018
Processing Suspected cases for 2019
Processing Suspected cases for 2020
Processing Suspected cases for 2021
Processing Suspected cases for 2022
Processing Suspected cases for 2023
Processing Suspected cases for 2024
Processing Suspected cases for 2025
GIF created: suspected_chikungunya_cases.gif


In [27]:
from google.colab import files

# Download the confirmed cases GIF
files.download('confirmed_chikungunya_cases.gif')

# Download the suspected cases GIF
files.download('suspected_chikungunya_cases.gif')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>